In [1]:
# Use autoplot 207's code for this app
import sys
sys.path.insert(0, "/opt/iem/htdocs/plotting/auto/scripts200/")
from p207 import load_data, compute_grid_bounds, add_zeros, do_analysis, USEME
from pyiem.plot import MapPlot, nwssnow
from pyiem.util import get_dbconn, utc
import numpy as np

In [18]:
STORM_NUMBER = 25
TITLE = "19-20 April 2021"
SUBTITLE = "8 AM 21 April 2021"
sts = utc(2021, 4, 18, 0)
ets = utc(2021, 4, 22, 17, 0)
# Get available data
ctx = {'coop': 'yes', 'csector': 'IA', 't': 'state', 'sz': 50, 'z': 'yes', 'f': 'linear', 'v': 'snow'}
df = load_data(ctx, sts, ets)
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx)
# add zeros and QC
df = add_zeros(df, ctx)

In [12]:
def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE, ),
        subtitle=("Snowfall totals till %s from NWS COOP, LSR, CoCoRaHS Reports; "
                  "IEM 2020-2021 Winter Storm #%s") % (SUBTITLE, STORM_NUMBER),
        twitter=True,
    )
    mp.contourf(
        lons, lats, vals, np.array([0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]), cmap=nwssnow(), clip_on=True)
    df_useme_plot = df2[(df2['val']>=lower)&(df2['val'] < upper)]
    print(df_useme_plot)
    mp.drawcounties()
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(df_useme_plot['lon'], df_useme_plot['lat'], df_useme_plot['val'],
            '%s', labels=df_useme_plot['nwsli'].values,
            textsize=10, labeltextsize=10, labelbuffer=1)
    return mp

In [23]:
cull = ['LSXI4', 'IA-PK-73', 'SXRI4', 'ROKI4', 'PEAI4', 'IA-DV-3', 'MMPM7', 'SACI4', 'IA-ID-4']
if cull:
    df.loc[df['nwsli'].isin(cull), USEME] = False
# df.loc[df[df["val"] < 1].index, USEME] = False
hardcode = [

]
for nwsli, val in hardcode:
    df.loc[df['nwsli'] == nwsli, 'val'] = val

ctx['csector'] = "IA"
mp = workflow(ctx, df, isfinal=True, lower=0, upper=4)
res = mp.postprocess(filename='210421.png')
mp.close()

     state  wfo  val         lon        lat                              geo  \
589     OH  ILN  3.9  -83.370000  40.240000  POINT (1400964.974 -387370.303)   
590     CO  BOU  3.9 -104.930000  39.920000  POINT (-420535.815 -552167.774)   
591     SD  UNR  3.9 -103.870000  44.510000   POINT (-306733.960 -47173.246)   
592     SD  ABR  3.9  -97.030000  44.730000    POINT (234554.160 -25728.042)   
593     CO  BOU  3.9 -105.490000  39.020000  POINT (-474502.271 -648976.833)   
...    ...  ...  ...         ...        ...                              ...   
4395    SD  UNR  0.0 -101.470000  44.870000   POINT (-115835.265 -13405.140)   
4396    TN  HUN  0.0  -86.215000  35.342200  POINT (1247954.769 -970472.719)   
4397    TN  MEG  0.0  -89.258600  35.589400   POINT (971339.067 -983324.248)   
4398    TN  MEG  0.0  -89.803900  35.129400  POINT (927928.953 -1040196.792)   
4399    TN  MEG  0.0  -88.727143  35.933035  POINT (1014402.493 -939167.338)   

      used_for_analysis  nwsli  plotme 